In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf

In [3]:
path = 'shakespeare.txt'

In [7]:
txt = open(path, 'r').read()

In [9]:
len(txt)

5445609

In [10]:
type(txt)

str

In [14]:
print(txt[10000:10300])

 sullied night,
    And all in war with Time for love of you,
    As he takes from you, I engraft you new.


                     16  
  But wherefore do not you a mightier way
  Make war upon this bloody tyrant Time?
  And fortify your self in your decay
  With means more blessed than my barren rhy


In [18]:
len(set(txt))

84

In [23]:
#sorted(set(txt))

In [27]:
characters = sorted(set(txt))

In [30]:
char_to_index = {char:ind for ind, char in enumerate(characters)} 

In [36]:
char_to_index['s']

74

In [42]:
index_to_char = np.array(characters)

In [45]:
index_to_char[70]

'o'

In [46]:
t = ' i am student'

In [47]:
[i for i in t]

[' ', 'i', ' ', 'a', 'm', ' ', 's', 't', 'u', 'd', 'e', 'n', 't']

### now coverting all txt into the numeric


In [49]:

coded_text = np.array([char_to_index[c] for c in txt])

In [51]:
len(coded_text)

5445609

## Creating batches